In [1]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager

from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
import torch
torch.manual_seed(0)

import random
random.seed(0)

import numpy as np
np.random.seed(0)

In [3]:
# Set patient and record (same channels)

observations = ["chb01_03", "chb01_04", "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
# observations = ["chb03_01", "chb03_02", "chb03_03", "chb03_04", "chb03_34", "chb03_35", "chb03_36"]

# observations = ["chb02_16", "chb02_19"]

# observations = ["chb04_28"]
# observations = ["chb05_06", "chb05_13", "chb05_16", "chb05_17", "chb05_22"]
# observations = ["chb06_01", "chb06_04"] #, "chb06_09", "chb06_10", "chb06_13", "chb06_18", "chb06_24"]

# observations = ["PN00_1", "PN00_2", "PN00_4", "PN00_5"]
# observations = ["PN14_1", "PN14_2", "PN14_3", "PN14_4"]
# observations = ["PN14_4"]
# observations = ["chb04_28"]

create_dataset_json(observations)

In [4]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache dataset

In [5]:
remove_cache_dataset = True

if remove_cache_dataset:
    folder = ".\data\cache\datasets"

    for file in os.listdir(folder):
        if file.startswith("GCS-"):  # Checks if the file begins with "GCS-"
            cache_path = os.path.join(folder, file)
            if os.path.isfile(cache_path):
                os.remove(cache_path)
                print(f"Removed: {cache_path}")

Removed: .\data\cache\datasets\GCS-7461d1da0d4c1403016b336840abe8b6


## Run Gretel experiment

In [6]:
# NOTICE: If it crashes, check if there are any OracleTorch files from the future!

In [7]:
path = 'config/GCS-GCN.jsonc'
# path = 'config/GCS-SVM.jsonc'
# path = 'config/GCS-KNN.jsonc'

print(observations)

print(f"Generating context for: {path}")
context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info(
    "Creating the evaluation manager......................................................."
)

if 'doe-triplets' in context.conf:
    context.logger.info("Creating the TRIPLET evaluators........................................................")
    eval_manager = TripletsEvaluatorManager(context)
if 'do-pairs' in context.conf:
    context.logger.info("Creating the PAIRED evaluators...............................................................") ### Usiamo questo
    eval_manager = PairedEvaluatorManager(context)
else:
    context.logger.info("Creating the evaluators...............................................................")
    eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)

eval_manager.evaluate()

# Additional stuff
print('--------------------------------------')

for i in range(len(eval_manager._evaluators)):
    # context.logger.info(f"graph_ids: {eval_manager._evaluators[i].get_instance_and_counterfactual_ids()}") # NON È STATO SISTEMATO PER CONTEMPLARE IL CASO DI PIU' PAZIENTI
    context.logger.info(f"graph_metrics: {eval_manager._evaluators[i].get_instance_and_counterfactual_graph_metrics()}")

['chb01_03', 'chb01_04', 'chb01_15', 'chb01_16', 'chb01_18', 'chb01_21', 'chb01_26']
Generating context for: config/GCS-GCN.jsonc
2025-04-03 17:26:26,514 | INFO | 8716 - Executing: config/GCS-GCN.jsonc Run: -1
2025-04-03 17:26:26,530 | INFO | 8716 - Creating the evaluation manager.......................................................
2025-04-03 17:26:26,570 | INFO | 8716 - Creating the PAIRED evaluators...............................................................
2025-04-03 17:26:26,653 | INFO | 8716 - Creating: GCS-7461d1da0d4c1403016b336840abe8b6
2025-04-03 17:26:26,655 | INFO | 8716 - Instantiating: src.dataset.generators.gcs.GCS
2025-04-03 17:26:34,669 | INFO | 8716 - Saved: GCS-7461d1da0d4c1403016b336840abe8b6
2025-04-03 17:26:34,696 | INFO | 8716 - Created: GCS-7461d1da0d4c1403016b336840abe8b6
2025-04-03 17:26:34,742 | INFO | 8716 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-04-03 17:26:34,752 | INFO | 8716 - Instantiating: torch.optim.RMSprop
2025-04-03 17:26:34,753 

KeyboardInterrupt: 

In [ ]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [ ]:
import datetime

now = datetime.datetime.now()
print("Last full run:", now.strftime("%d/%m/%Y, ore %H:%M"))

Last full run: 03/04/2025, ore 15:28


In [ ]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)

In [ ]:
# from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

# results_path = "output\\results"
# stats_file_path = "output"
# res = data_analyzer.create_aggregated_dataframe(results_path)
# res.to_csv(stats_file_path)
# res